In [1]:
import random

import os
import pygad

cwd = os.getcwd()
pygad_loc = '/grid/home/nbourgeois/codonOpt'
os.chdir(pygad_loc)
from general_functions import *
from metrics import *
os.chdir(cwd)
import seaborn as sns

Using TensorFlow backend.


In [2]:
tissue = 'Liver'
cai_weights = get_codon_weights(tissue)
bai_weights = get_bicodon_weights(tissue)
codon_usage_table_loc = os.path.join(pygad_loc,'references','codon_usage.getex.txt')

In [5]:
def get_back_table(codon_usage_table_loc=os.path.join(pygad_loc,'references/codon_usage.getex.txt')):
    codon_usage_table = pd.read_csv(codon_usage_table_loc,sep='\t')
    forward_table = pd.Series(codon_usage_table.AA.values,index=codon_usage_table.Codon).to_dict()

    back_table = {}
    for key in forward_table:
        val = forward_table[key]
        if val not in back_table.keys():
            back_table[val] = [key]
        else:
            back_table[val].append(key)

    codon_to_int = {}
    i=0
    for codon in forward_table.keys():
        codon_to_int[codon] = i
        codon_to_int[i] = codon
        i += 1
    return(back_table)

def get_codon_space(aa_seq,back_table):
    gene_space = []
    for aa in aa_seq:
        all_cds = back_table[aa]
        gene_space.append(all_cds)
    return(gene_space)

def gen_rand_aa(n):
    rand_aas = max(n-2,0)
    aa_list = ['A','R','N','D','C','Q','G','E','H','I',
               'L','K','F','P','S','T','W','Y','V']
    index_list = np.random.randint(19,size=rand_aas)
    aa_seq = 'M'+''.join([aa_list[ind] for ind in index_list]) + '*'
    return(aa_seq)
def gen_random_seqs(codon_space, num_seqs):
    positions = len(codon_space)
    size_vector = [len(codon_pos) for codon_pos in codon_space] 
    seqs = [None]*num_seqs
    for i in range(num_seqs):
#         rand_index = [random.randrange(size) for size in size_vector]
        rand_index = [int(size * random.random()) for size in size_vector]
        seqs[i] = ''.join(([space[ind] for ind, space in zip(rand_index,codon_space)]))
    return(seqs)





ERROR! Session/line number was not unique in database. History logging moved to new session 3025


In [16]:
bai_weights

{'TTTTTT': 0.14153010255438922,
 'TTTTTC': 0.20508768693905877,
 'TTTTTA': 0.07025506134522225,
 'TTTTTG': 0.1836241280217444,
 'TTTCTT': 0.12295864397899496,
 'TTTCTC': 0.30842837751453855,
 'TTTCTA': 0.12459963155290636,
 'TTTCTG': 0.20639642532515187,
 'TTTATT': 0.43274498921346305,
 'TTTATC': 0.24082606193892556,
 'TTTATA': 0.11269123567843446,
 'TTTATG': 0.230578789855782,
 'TTTGTT': 0.36570411060364455,
 'TTTGTC': 0.5640140964258903,
 'TTTGTA': 0.3340564182846646,
 'TTTGTG': 1.0,
 'TTTTAT': 0.527602986974566,
 'TTTTAC': 0.41296852878950346,
 'TTTTAA': 0.31099687739350734,
 'TTTTAG': 0.36059624108878807,
 'TTTCAT': 0.28236016519909607,
 'TTTCAC': 0.26449684407387203,
 'TTTCAA': 0.09727966223525385,
 'TTTCAG': 0.3045182476831808,
 'TTTAAT': 0.43798565801349804,
 'TTTAAC': 0.31351127694232994,
 'TTTAAA': 0.5171822518864788,
 'TTTAAG': 0.3741723142667354,
 'TTTGAT': 1.0,
 'TTTGAC': 0.6959354306910741,
 'TTTGAA': 1.0,
 'TTTGAG': 0.6934061169424125,
 'TTTTCT': 0.33703712371546535,
 'TT

In [28]:
min_val=1
for key in bai_weights.keys():
    val = bai_weights[key]
    if val < min_val and val!=0:
        if key.startswith('TAA') or key.startswith('TAG') or key.startswith('TGA'):
            continue
        min_val = val
        min_key = key
        print(min_val,min_key)
bai_weights[min_key]
min_key

0.14153010255438922 TTTTTT
0.07025506134522225 TTTTTA
0.021750139277189714 TTTACG
0.01699478016243818 TTATTA
0.013924208091585955 TTACTA
0.004466296066981232 TTATCG


'TTATCG'

In [51]:
min_val=1
for key in cai_weights.keys():
    val = cai_weights[key]
    if val < min_val and val!=0:
        if key.startswith('TAA') or key.startswith('TAG') or key.startswith('TGA'):
            continue
        min_val = val
        min_key = key
        print(min_val,min_key)
cai_weights[min_key]
min_key

0.75 TTT
0.13942307692307693 TTA


'TTA'

ERROR! Session/line number was not unique in database. History logging moved to new session 3029


In [54]:
aa_seq = gen_rand_aa(150)

codon_to_int, codon_space = init_parameters_fun(aa_seq,codon_usage_table_loc)
   
seqs = gen_random_seqs(codon_space,100000)

In [55]:
diff_bai = []
diff_cai = []
for seq in seqs:
    bad_seq = seq[:-9] + 'TTATCG' + 'TAG'
    diff_bai.append(get_bai(seq,bai_weights)- get_bai(bad_seq,bai_weights))
    diff_cai.append(get_cai(seq,cai_weights)- get_cai(bad_seq,cai_weights))

In [56]:
max(diff_bai),max(diff_cai)

(0.021239962241138888, 0.01660810350265851)